Initialize Gen and the function libraries

In [1]:
using Gen

In [19]:
cd("/Users/lilis/Documents/GitHub/HSOManalysisPackages/JuliaLanguage_Hierarchical_Models")
include(join([pwd(), "/function_library/file_modules.jl"]))
refresh_tools("function_library")
refresh_tools("gen_stepvramp_library")

Refreshed (2021-03-26_1800) GLMNet_tools.jl auto_pull_matlabdatasets.jl cluster_modules.jl feedback_modules.jl file_modules.jl hazard_modules.jl v 11:37
math_modules.jl modeling_tools.jl plotting_tools.jl probabilistic_modules.jl Refreshed (2021-03-26_1800) gen_math_modules.jl gen_plotting_modules.jl gen_stepvramp.jl runner_modules.jl svr_filemods.jl 

Enter the CSV folder for all our sessions

In [9]:
# cd("N:\\Assad Lab\\allison\\Collated Analyses\\JULIA_CSVs\\All_SNc_nosplitsesh")
cd("/Volumes/Neurobio/Assad Lab/allison/Collated Analyses/JULIA_CSVs/All_SNc_nosplitsesh")

In [22]:
B1_idx = 1:5
B2_idx = 6:13
B3_idx = 14:24
B5_idx = 75:80
B6_idx = 25:30
H3_idx = 44:51
H4_idx = 52:66
H5_idx = 67:74
H6_idx = 81:92
H7_idx = 93:98
H14_idx = 31:38
H15_idx = 39:43

39:43

In [ ]:
runID = "2021_3_26_Mac"
pathIDx = [75]
# mac
collatedPath = "/Volumes/Neurobio/Assad Lab/allison/Collated Analyses/JULIA_CSVs/All_SNc_nosplitsesh"
compositesavepath="/Volumes/Neurobio/Assad Lab/allison/Collated Analyses/HIERARCHY results/3-26-21 hierarchy1"
# windows
# collatedPath = "N:\\Assad Lab\\allison\\Collated Analyses\\JULIA_CSVs\\All_SNc_nosplitsesh"
# compositesavepath="N:\\Assad Lab\\allison\\Collated Analyses\\HIERARCHY RESULTS\\3-26-21 hierarchy1"
modelpackagefunction = hierarchy_v1#SHUFFLE_DAHxmodel_logit_200hx_pkg
postprocessingfunction = template_postprocessingfunction#bootlogit_timeslice_postprocessingfunction1

(fails, results,postprocessing) = run_collated_model(collatedPath, 
    modelpackagefunction; pathIDx = pathIDx,runFails=false, failDirs=[], postprocessingfunction=postprocessingfunction,
    compositesavepath=compositesavepath, runID=runID, suppressFigures=true); 